# Computing distances between stations

In this section, we compute the distances between the stations that form the transsects. This will be useful to then plot vertical sections where the x axes will be the distance between stations. 

In [4]:
#Imports
import sys
import numpy as np
import pandas as pd
from shapely.geometry import Point, LineString, MultiPoint

#Import path to custom functions
sys.path.append('C:/Users/pauab/Universidad/aProgramacion/TFM/Notebooks_Pau/my_functions/')
#Import custom functions
from read_CTD import read_CTD    #To read CTD files
from out_outliers import out_outliers     #To replace outliers

#Import path to custom functions
sys.path.append('C:/Users/pauab/Universidad/aProgramacion/TFM/Notebooks_Pau/my_functions/distance/')
from distances import *

#To turn off warnings
import warnings
warnings.filterwarnings('ignore')

In [7]:
#Data reading and outlier replacement
data_dirs = ['C:/Users/pauab/Universidad/Data/TFM/CTD_recalc/']
data=read_CTD(data_dirs)

for ncast in data.keys():
    for variable in data[ncast].columns:
        out_outliers(data, ncast, variable)
    data[ncast]['pressure']=data[ncast].index
    data[ncast].index = np.arange(0, len(data[ncast].index))
    
#Dict with all transsects and ncasts
transsects = {'transsect_0':['15','12','13','14'],
              'transsect_1':['19','18','17','16'],
              'transsect_2':['24','23','22','21'],
              'transsect_3':['26','27','28','29'],
              'transsect_4':['31','32','33','34'],
              'transsect_5':['38','37','36','35'],
              'transsect_6':['43','44','45','46']}

In [8]:
# The following code computes the distances of the stations that form a transect
# It stores the distances in a dict where the keys are the transects' names and the values the distances between stations

distances = {}
#Iterate over the transects where you want to compute the distances
for transsect in transsects.keys():
    #Get the lats and lons of stations
    station_lats = []
    station_longs = []
    for ncast in transsects[transsect]:
        station_lats.append(data[ncast]['latitude'].iloc[-1])
        station_longs.append(data[ncast]['longitude'].iloc[-1])
        
    # Define temp variable to store results
    tmp_dist = []

    # Iterate over lats and longs and number of stations to compute distances
    for i, j, k in zip(station_lats, station_longs, range(len(station_lats))):

        # First distance from section_start (xoffset) and first ctd station
        if k == 0:
            tmp_dist.append(dist_aux(station_lats[0], station_longs[0], i, j))
            tmp_lat = i
            tmp_lon = j
        else:
            tmp_dist.append(dist_aux(tmp_lat, tmp_lon, i, j))
            tmp_lat = i
            tmp_lon = j

    # Perform cumulative summatory
    tmp_dist = list(np.cumsum(tmp_dist))
    
    #Add distances to the dictionary
    distances[transsect] = tmp_dist

We created the function transsect_distances.py to simplify the previous code. The function computes the distances between the stations of n given transects.

It returns a dict with the lists of the distances (values) of each transsect (keys).